# Trabajo Final procesamiento de lenguaje natural (NLP) <a class="tocSkip">
## Universidad Pontificia Bolivariana <a class="tocSkip">

**Estudiante:** Valentina Palacio Castañeda

**ID:**XXXXXX



**Subir Este notebook de Trabajo Final en su repositorio de GitHub. y enviar el link en **Microsoft TEAMS**

Docente: [Jose R. Zapata](https://joserzapata.github.io)
- https://joserzapata.github.io
- https://www.linkedin.com/in/jose-ricardo-zapata-gonzalez/       


## Objetivo del Trabajo
Realizar un proceso de Procesamiento de lenguaje natural (NLP) para dejar los datos preparados para ser usados con algoritmos de Machine Learning para Clasificación como objetivo final del trabajo.

los datos preparados para ser usados con algoritmos de Machine Learning para Regresión o Clasificación como objetivo final del trabajo.

El trabajo se realizara en este jupyter notebook y subirlo a su repositorio de github creado en clase. (**Recuerde poner su nombre e información**)

## Las actividades a realizar
    

1) Limpiar los datos de texto.

   - https://joserzapata.github.io/courses/nlp/procesamiento-basico/

   - https://joserzapata.github.io/courses/nlp/preprocesamiento-texto/

2) Realizar la representación de texto:

   - https://joserzapata.github.io/courses/nlp/representaciones/

   - Tokenización
   - Lematización o stemming
   - Representación de los datos de texto (Bag of Words o TF-IDF)

3) Utilice un modelo de Machine Learning para clasificación para entrenar y evaluar el modelo.
        
    - Calcule estas las métricas de evaluación (accuracy, precision, recall, f1-score) - https://joserzapata.github.io/courses/python-ciencia-datos/clasificacion/#evaluacion-modelo-simple


    
   
    
*NOTA: No dude en contactarme para cualquier pregunta o inquietud :) por el chat de Teams o al correo
joser.zapata@upb.edu.co*

## EVALUACIÓN


|Porcentaje en la evaluación | Descripción| Nada | Incompleto | Completo
| :---: |:---: |:---: |:---: |:---:
| 5 % |**limpieza de los datos** |
| 5 % | **Representacion de los datos** |
| 5 % | **Machine Learning** <br> Entrenar y evaluar el modelo propuesto |


In [1]:
import pandas as pd

from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
    classification_report,
)
from sklearn.model_selection import train_test_split

Carga del dataset

El archivo  se carga en aproximadamente 10 segundos

In [3]:
# Usar su ID Ejemplo: "0028984798"
id_ = "000580259"

data_reviews = pd.read_parquet(
    "https://www.dropbox.com/scl/fi/gvk9yj8cn96oocr9z058x/filmaffinity_reviews_dataset.parquet?rlkey=xgvr00zvkxbkwqqavqutpsshg&st=xjb7xze9&dl=1"
)
data_reviews = data_reviews.sample(n=50_000, random_state=int(id_upb))
data_reviews.info()

<class 'pandas.core.frame.DataFrame'>
Index: 50000 entries, 16165 to 75984
Data columns (total 2 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   author_review_desc  50000 non-null  string
 1   author_rating       50000 non-null  int64 
dtypes: int64(1), string(1)
memory usage: 1.1 MB


Ejemplo de algunas filas del dataset

In [4]:
data_reviews.sample(5)

,author_review_desc,author_rating
2945,"Despuès del reconocimiento de su antecesora, ...",6
78817,Pocas veces me ha impactado tanto un personaj...,9
93626,"La serie empieza con una premisa interesante,...",5
61680,"Junto al tan usado calificativo de ""entreteni...",4
29551,Secuela obligada de la genial primera parte q...,6


Evaluar los valores nulos

In [5]:
data_reviews.isnull().sum()

,0
author_review_desc,0
author_rating,0


En este caso no hay nulos.

Revisar si hay valores duplicados:

In [6]:
data_reviews.duplicated().sum()

np.int64(3)

No hay  valores duplicados.

## Clasificación Tradicional para Análisis de Sentimientos y Categorías 👍 👎



Definición: Se considera reseña positiva cuando la puntuación ("author_rating") es mayor que 6; negativa en caso contrario.

Crear la variable binaria de sentimiento: 1 (positivo) si author_rating > 6, 0 (negativo) de lo contrario

In [7]:
UMBRAL = 6
data_reviews["sentiment_bin"] = (data_reviews["author_rating"] > UMBRAL).astype(int)

In [8]:
# Dividir los datos en conjuntos de entrenamiento y prueba

X_data = data_reviews["author_review_desc"]
y_data = data_reviews["sentiment_bin"]


Dividir el dataset en entrenamiento y prueba

In [9]:
X_train, X_test, y_train, y_test = train_test_split(
    X_data,
    y_data,
    test_size=0.2,
    stratify=y_data,  # Mantener la proporción de clases en ambos conjuntos
    random_state=42,
)

## Análisis exploratorio de los datos

para determinar que tipo de limpieza se debe realizar a los datos de texto de `X_train`

In [10]:
#Contar palabras promedio por reseña**

X_train.apply(lambda x: len(x.split())).describe()

,author_review_desc
count,40000.000000
mean,237.061525
std,179.826839
min,5.000000
25%,108.000000
50%,185.000000
75%,313.000000
max,1106.000000


In [12]:
### Análisis exploratorio de longitud de reseñas

#El análisis muestra que las reseñas tienen una longitud promedio de 237 palabras, con una gran variabilidad (std ≈ 180). Algunas reseñas son muy extensas (hasta 1106 palabras), mientras que otras son muy breves (mínimo de 5 palabras).

#Decisiones de limpieza:**

 #Convertir texto a minúsculas para normalizar.
 #Eliminar puntuación, números y símbolos innecesarios.
 #Eliminar stopwords en español para reducir ruido.
 #Normalizar espacios y eliminar URLs.

## 1) Limpieza de los datos de texto

Tomar los datos de `X_train` y aplicar las funciones de limpieza que considere necesarias

Ayudas:

- Convertir a minúsculas
- Eliminar caracteres especiales y números
- Corregir palabras mal escritas
- etc


In [13]:
import nltk
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [14]:
import re
from nltk.corpus import stopwords

stopwords_es = set(stopwords.words('spanish'))

def limpiar_texto(texto):
    texto = texto.lower()  # minúsculas
    texto = re.sub(r"http\S+|www\S+|https\S+", '', texto)  # eliminar URLs
    texto = re.sub(r"[^a-záéíóúñü\s]", '', texto)  # eliminar puntuación y símbolos
    texto = re.sub(r"\d+", '', texto)  # eliminar números
    texto = re.sub(r"\s+", ' ', texto).strip()  # normalizar espacios
    palabras = texto.split()
    palabras = [p for p in palabras if p not in stopwords_es]  # eliminar stopwords
    return ' '.join(palabras)


In [15]:
X_train_clean = X_train.apply(limpiar_texto)
X_test_clean = X_test.apply(limpiar_texto)


## 🧹 Limpieza de los datos de texto

Se aplican las siguientes transformaciones a `X_train` y `X_test`:

- Conversión a minúsculas
- Eliminación de URLs, puntuación, símbolos y números
- Eliminación de stopwords en español
- Normalización de espacios

Estas acciones permiten reducir el ruido y mejorar la representación semántica del texto.

## 2) Representacion del texto

Luego de tener los datos limpios, realizar la representación de los datos de texto para poder usarse en modelos de machine learning.

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features=5000, ngram_range=(1,2))
X_train_vec = vectorizer.fit_transform(X_train_clean)
X_test_vec = vectorizer.transform(X_test_clean)


## 🧬 Representación vectorial del texto

Se utiliza `TfidfVectorizer` para transformar las reseñas limpias en vectores numéricos. Esta técnica pondera las palabras según su frecuencia en el documento y en el corpus, lo que mejora la capacidad del modelo para identificar patrones relevantes.

## 3) Entrenar un modelo de machine learning de clasificación

Utilizar un modelo de clasificación para entrenar y evaluar el modelo con los datos preparados.


In [17]:
from sklearn.linear_model import LogisticRegression

modelo = LogisticRegression(max_iter=1000)
modelo.fit(X_train_vec, y_train)
y_pred = modelo.predict(X_test_vec)


## 🤖 Entrenamiento del modelo de clasificación

Se entrena un modelo de clasificación (`LogisticRegression`) usando los vectores TF-IDF como entrada y la variable binaria de sentimiento como salida.

Este modelo permite predecir si una reseña es positiva o negativa según su contenido textual.


### Evaluar el modelo con los datos de prueba

Usar el modelo para predecir en `X_test` y evaluar con `y_test`

**Nota:** Recuerde que `X_test` debe pasar por los mismos procesos de limpieza y representación que `X_train`

## 📊 Evaluación del modelo

Se evalúa el rendimiento del modelo sobre el conjunto de prueba usando las siguientes métricas:

- Accuracy
- Precision
- Recall
- F1 Score
- ROC AUC

Estas métricas permiten medir la calidad de las predicciones y el balance entre falsos positivos y falsos negativos.


In [18]:
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred))
print("Recall:", recall_score(y_test, y_pred))
print("F1 Score:", f1_score(y_test, y_pred))
print("ROC AUC Score:", roc_auc_score(y_test, y_pred))
print("Reporte de clasificación:\n", classification_report(y_test, y_pred))

Accuracy: 0.8012
Precision: 0.8019340974212035
Recall: 0.8354477611940299
F1 Score: 0.8183479532163743
ROC AUC Score: 0.7985428461142563
Reporte de clasificación:
               precision    recall  f1-score   support

           0       0.80      0.76      0.78      4640
           1       0.80      0.84      0.82      5360

    accuracy                           0.80     10000
   macro avg       0.80      0.80      0.80     10000
weighted avg       0.80      0.80      0.80     10000



**Guardar el modelo**

In [19]:
import joblib

In [24]:
joblib.dump(modelo, "modelo_sentimientos_filmaffinity.pkl")
joblib.dump(vectorizer, "vectorizer_tfidf.pkl")

['vectorizer_tfidf.pkl']

## 💾 Guardado del modelo y vectorizador

Se guarda el modelo entrenado y el vectorizador TF-IDF en archivos `.pkl` para su reutilización futura. Esto permite aplicar el modelo a nuevos datos sin necesidad de reentrenar.


## Librerías Usadas

In [22]:
!pip install watermark

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 70.3 MB/s eta 0:00:00


In [23]:
from watermark import watermark

print(watermark(python=True, iversions=True, globals_=globals()))

Python implementation: CPython
Python version       : 3.12.12
IPython version      : 7.34.0

joblib   : 1.5.2
watermark: 2.5.0
re       : 2.2.1
pandas   : 2.2.2
sklearn  : 1.6.1
nltk     : 3.9.1



---

## Referencias
- [CountVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html)
- [TfidfVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html)
- [Classification of text documents using sparse features](https://scikit-learn.org/stable/auto_examples/text/plot_document_classification_20newsgroups.html)
- Ejemplo de Entrenamiento y selección de Modelo de machine learning entre varios modelos <https://joserzapata.github.io/post/ciencia-datos-proyecto-python/6-model_selection/>
- https://joserzapata.github.io/courses/python-ciencia-datos/python/
- https://joserzapata.github.io/courses/python-ciencia-datos/pandas/
- https://joserzapata.github.io/courses/python-ciencia-datos/machine-learning/

Docente: [Jose R. Zapata](https://joserzapata.github.io)

- [https://joserzapata.github.io/](https://joserzapata.github.io/)
- [https://www.linkedin.com/in/jose-ricardo-zapata-gonzalez/](https://www.linkedin.com/in/jose-ricardo-zapata-gonzalez/)